In [2]:
import torch
import torch.nn as nn
from torchvision import models, transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Function

In [4]:
# GradCAM 클래스 정의
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None

        # 타겟 레이어에서 기울기 값을 얻기 위해 Hook 설정
        self.hook_layer()

    # Forward 및 Backward에서 Hook을 설정하는 함수
    def hook_layer(self):
        # Forward 시 활성화 값 저장
        def forward_hook(module, input, output):
            self.activation = output
        
        # Backward 시 기울기 값 저장
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0]
        
        # Forward 및 Backward에서 Hook 등록
        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    # 모델에 입력 데이터를 전달하는 함수
    def forward(self, x):
        return self.model(x)

    # GradCAM 결과를 생성하는 함수
    def generate_cam(self, input_image, target_class):
        # 기울기 계산을 위해 모델의 기울기를 0으로 초기화
        self.model.zero_grad()
        output = self.forward(input_image)

        # 타겟 클래스에 대한 점수 추출
        target = output[:, target_class]
        target.backward()  # 역전파 수행

        # 기울기의 평균 계산
        pooled_gradients = torch.mean(self.gradients, dim=[0, 2, 3])

        # 기울기를 기반으로 활성화 맵에 가중치 적용
        for i in range(self.activation.shape[1]):
            self.activation[:, i, :, :] *= pooled_gradients[i]

        # 채널별로 평균을 내어 최종 CAM 생성
        cam = torch.mean(self.activation, dim=1).squeeze()

        # 음수 값을 제거하기 위해 ReLU 적용
        cam = torch.clamp(cam, min=0)

        # CAM을 0과 1 사이의 값으로 정규화
        cam = (cam - cam.min()) / (cam.max() - cam.min())
        return cam.detach().cpu().numpy()

In [5]:
# 이미지 전처리 함수
def preprocess_image(image_path):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((512, 512)),  # 입력 이미지 크기를 512x512로 조정
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 표준화 적용
    ])
    # 이미지 읽기
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 변환
    input_tensor = preprocess(image).unsqueeze(0)  # 배치 차원을 추가
    return image, input_tensor

# GradCAM 결과를 시각화하는 함수
def show_cam_on_image(image, mask):
    # CAM 마스크를 컬러맵으로 변환
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255  # 정규화
    cam = heatmap + np.float32(image) / 255  # 원본 이미지와 합성
    cam = cam / np.max(cam)  # 값을 0~1 사이로 정규화
    return np.uint8(255 * cam)  # 다시 255 범위로 변환

In [ ]:
# 모델 로드 및 GradCAM 적용 함수
# 이미지 경로와 타겟 클래스를 지정하고 GradCAM 실행
image_path = 'path_to_your_image.jpg'  # 분석할 이미지 경로
target_class = 1  # 예: 0 - 이물질 없음, 1 - 이물질 종류 1, 2 - 이물질 종류 2

# 사전 학습된 ResNet18 모델 로드 (다중 클래스 분류를 위한 ResNet18)
model = models.resnet18(pretrained=True)

# 마지막 레이어를 이물질 분류에 맞게 수정 (3개의 클래스: 이물질 없음, 이물질 종류 1, 이물질 종류 2)
model.fc = nn.Linear(model.fc.in_features, 3)
model.eval()  # 모델을 평가 모드로 설정

# 타겟 레이어는 ResNet18의 마지막 합성곱 층 (layer4의 두 번째 블록)
target_layer = model.layer4[1].conv2

# GradCAM 초기화
grad_cam = GradCAM(model, target_layer)

# 이미지 전처리
raw_image, input_tensor = preprocess_image(image_path)

# GradCAM 결과 생성
cam_mask = grad_cam.generate_cam(input_tensor, target_class)

# CAM 마스크를 원본 이미지 크기에 맞게 리사이즈
cam_mask = cv2.resize(cam_mask, (raw_image.shape[1], raw_image.shape[0]))

# CAM 결과를 원본 이미지 위에 겹쳐서 시각화
cam_image = show_cam_on_image(raw_image, cam_mask)

# 결과 출력
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.imshow(raw_image)
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(cam_image)
plt.title('GradCAM')
plt.show()